# Text Generation with Phi-3
This notebook introduces basic text generation using a modern instruction-tuned LLM: Microsoft Phi-3.
It demonstrates how temperature and sampling settings impact creativity in generated responses.

### Step 1: Install Required Libraries (Only for Colab or Local Setup)
```python
# !pip install torch==2.3.1 transformers==4.41.2 sentence-transformers==3.0.1 \
#     matplotlib==3.9.0 scikit-learn==1.5.0 sentencepiece==0.2.0 \
#     nltk==3.8.1 evaluate==0.4.2 scipy==1.15.0
```

In [ ]:
from huggingface_hub import login
login(token="your_huggingface_token_here")  # Replace with your actual token

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Loading Phi-3 model...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

Loading Phi-3 model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:05<00:00,  2.88s/it]
Some parameters are on the meta device because they were offloaded to the disk.


**About the model:**
Phi-3 Mini is a small instruction-tuned LLM by Microsoft, optimized for fast and efficient inference.
Despite its small size (~1.3B parameters), it performs very well thanks to highly curated training data.

In [3]:
messages = [
    {'role': 'user', 'content': 'Tell me a funny joke about cats'}
]

In [4]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=100,
    do_sample=False
)

print("\n[Deterministic Output — do_sample=False]")
output = generator(messages)
print(output[0]["generated_text"])


[Deterministic Output — do_sample=False]


You are not running the flash-attention implementation, expect numerical differences.


 Why don't cats play poker in the jungle? Too many cheetahs!


### Step 2: Lower Temperature Sampling
- Temperature controls randomness:
  - Lower = safer, more predictable
  - Higher = more diverse, creative

In [6]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1
)

print("\n[Low Temperature Sampling — temperature=0.1] - probably the same joke")
output = generator(messages)
print(output[0]["generated_text"])  


[Low Temperature Sampling — temperature=0.1] - probably the same joke
 Why don't cats play poker in the jungle? Too many cheetahs!


### Step 3: Higher Temperature with Sampling Tricks
- `temperature`: how adventurous to be
- `top_k`: limit to top K tokens
- `repetition_penalty`: discourage repeats

In [7]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.6,
    top_k=50,
    repetition_penalty=1.2
)

print("\n[Creative Sampling — temperature=0.6, top_k=50, repetition_penalty=1.2]")
output = generator(messages)
print(output[0]["generated_text"])


[Creative Sampling — temperature=0.6, top_k=50, repetition_penalty=1.2]
 Why don't you ever play hide and seek with your cat?
Because good luck hiding when they find it by the smell of fish! (This brings humor to feline tracking skills, often associated in jokes.)


### Summary
- Deterministic = repeatable
- Sampling = more creativity
- Phi-3 = great mix of speed + quality

Try more prompts:
- "Explain transformers to a 5-year-old"
- "Write a haiku about machine learning"
- "What is RAG in large language models?"